In [ ]:
# ========= Invoice OCR → Excel (No OpenCV / No Paddle) =========
# Works with: PDF/PNG/JPG/TIFF
# Dependencies: pytesseract, PyMuPDF (fitz), Pillow, pandas, openpyxl, dateparser

import os
import re
import json
import pandas as pd
from io import BytesIO
from PIL import Image, ImageOps, ImageFilter
import fitz  # PyMuPDF
import pytesseract
import dateparser

# ---- ✅ FIXED: مسار tesseract.exe على جهازك
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# ---- 1) Helpers: PDF → PIL Images
def pdf_to_images(pdf_path, dpi=250):
    """Render PDF pages to PIL Images using PyMuPDF (no external poppler)."""
    images = []
    doc = fitz.open(pdf_path)
    for page in doc:
        pix = page.get_pixmap(dpi=dpi, alpha=False)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        images.append(img)
    doc.close()
    return images

# ---- 2) Preprocess image (خفيف بدون OpenCV)
def preprocess_image(img: Image.Image) -> Image.Image:
    # To grayscale
    g = ImageOps.grayscale(img)
    # Slight sharpen / contrast
    g = g.filter(ImageFilter.SHARPEN)
    # Simple binarization (adaptive-like via autocontrast then threshold)
    g = ImageOps.autocontrast(g)
    # Threshold
    g = g.point(lambda x: 255 if x > 160 else 0)
    return g

# ---- 3) OCR a single page
def ocr_page(img: Image.Image, lang="eng") -> str:
    # Try a couple of PSM modes for invoices
    configs = [
        "--oem 3 --psm 6",  # Assume a block of text
        "--oem 3 --psm 4",  # Assume a single column of text
    ]
    texts = []
    for cfg in configs:
        text = pytesseract.image_to_string(img, lang=lang, config=cfg)
        texts.append(text)
    # choose longer result (very simple heuristic)
    return max(texts, key=len)

# ---- 4) Parse amounts & fields
AMOUNT_RE = re.compile(r'[\$€£]?\s?\d{1,3}(?:[,\s]\d{3})*(?:\.\d{2})?')
def find_amount(line: str):
    m = AMOUNT_RE.search(line.replace(',', ''))
    if m:
        return m.group(0).strip()
    return None

def normalize_amount(val):
    if not val:
        return None
    v = val.replace(',', '').replace(' ', '')
    v = v.replace('EGP', '').replace('$','').replace('€','').replace('£','')
    try:
        return float(v)
    except:
        return None

def parse_invoice_text(text: str):
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    joined = "\n".join(lines)

    # Invoice number patterns
    inv_patterns = [
        r'Invoice\s*(No\.?|#)\s*[:\-]?\s*([A-Za-z0-9\-\/]+)',
        r'Inv(?:oice)?\s*[:\-]?\s*([A-Za-z0-9\-\/]+)',
        r'Invoice\s*ID\s*[:\-]?\s*([A-Za-z0-9\-\/]+)',
    ]
    invoice_no = None
    for pat in inv_patterns:
        m = re.search(pat, joined, flags=re.IGNORECASE)
        if m:
            invoice_no = m.group(len(m.groups()))
            break

    # Date: use dateparser to find first valid date
    date_val = None
    try:
        from dateparser.search import search_dates
        found = search_dates(joined, languages=['en'])
        if found:
            # pick first reasonable date
            for txt, dt in found:
                if 1990 <= dt.year <= 2100:
                    date_val = dt.strftime("%Y-%m-%d")
                    break
    except Exception:
        date_val = None

    # Try to detect currency from text
    currency = None
    if re.search(r'\bEGP\b|ج\.م|E£', joined, flags=re.IGNORECASE):
        currency = "EGP"
    elif re.search(r'USD|\$', joined, flags=re.IGNORECASE):
        currency = "USD"
    elif re.search(r'EUR|€', joined, flags=re.IGNORECASE):
        currency = "EUR"
    elif re.search(r'SAR', joined, flags=re.IGNORECASE):
        currency = "SAR"
    elif re.search(r'AED', joined, flags=re.IGNORECASE):
        currency = "AED"

    # Subtotal / Tax / Total (line heuristics)
    subtotal = None
    tax = None
    total = None

    for ln in lines:
        low = ln.lower()
        if subtotal is None and "subtotal" in low:
            subtotal = find_amount(ln)
        if tax is None and ("tax" in low or "vat" in low):
            tax = find_amount(ln)

    for ln in lines:
        low = ln.lower()
        if "grand total" in low or "total due" in low or re.search(r'\btotal\b', low):
            amt = find_amount(ln)
            if amt:
                total = amt

    # Vendor name
    vendor = None
    for ln in lines[:6]:
        if len(ln) > 2 and not re.search(r'invoice', ln, re.IGNORECASE):
            if re.search(r'[A-Za-z]{2,}', ln):
                vendor = ln
                break

    # Normalize numeric
    subtotal_f = normalize_amount(subtotal)
    tax_f = normalize_amount(tax)
    total_f = normalize_amount(total)

    consistent = None
    if subtotal_f is not None and tax_f is not None and total_f is not None:
        consistent = abs((subtotal_f + tax_f) - total_f) < 1e-2

    return {
        "vendor": vendor,
        "invoice_no": invoice_no,
        "date": date_val,
        "subtotal": subtotal_f if subtotal_f is not None else subtotal,
        "tax": tax_f if tax_f is not None else tax,
        "total": total_f if total_f is not None else total,
        "currency": currency,
        "consistent": consistent
    }

# ---- 5) Main: process a file path (PDF or image) and append to Excel
def process_invoice_to_excel(file_path: str, excel_path="extracted_invoices.xlsx"):
    ext = os.path.splitext(file_path)[1].lower()
    pages = []

    if ext == ".pdf":
        pages = pdf_to_images(file_path, dpi=250)
    else:
        pages = [Image.open(file_path).convert("RGB")]

    full_text = ""
    for img in pages:
        img_p = preprocess_image(img)
        page_txt = ocr_page(img_p, lang="eng")
        full_text += page_txt + "\n"

    fields = parse_invoice_text(full_text)

    row = {
        "file": os.path.basename(file_path),
        **fields
    }

    df_new = pd.DataFrame([row])

    if os.path.exists(excel_path):
        df_old = pd.read_excel(excel_path)
        df_all = pd.concat([df_old, df_new], ignore_index=True)
    else:
        df_all = df_new

    df_all.to_excel(excel_path, index=False)

    print("=== OCR RESULT ===")
    print(json.dumps(row, indent=2))
    print(f"\nSaved/updated Excel → {excel_path}")

    return row



In [3]:
result = process_invoice_to_excel(r"C:\Users\pc\Desktop\invoices_10_files\sample_invoice_01.pdf")
print(result)


=== OCR RESULT ===
{
  "file": "sample_invoice_01.pdf",
  "vendor": "Sample Vendor 1",
  "invoice_no": "INV-1001",
  "date": "2025-02-01",
  "subtotal": null,
  "tax": null,
  "total": null,
  "currency": "USD",
  "consistent": null
}

Saved/updated Excel → extracted_invoices.xlsx
{'file': 'sample_invoice_01.pdf', 'vendor': 'Sample Vendor 1', 'invoice_no': 'INV-1001', 'date': '2025-02-01', 'subtotal': None, 'tax': None, 'total': None, 'currency': 'USD', 'consistent': None}


C:\Users\pc\AppData\Local\Temp\ipykernel_13420\3623265658.py:192: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_old, df_new], ignore_index=True)
